In [2]:
# 한글 폰트 설정
# 한글 폰트 설치 코드
!apt-get update -qq
!apt-get install fonts-nanum* -qq
# 설치 후 런타임 재시작 필요

Selecting previously unselected package fonts-nanum.
(Reading database ... 155680 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20170925-1_all.deb ...
Unpacking fonts-nanum (20170925-1) ...
Selecting previously unselected package fonts-nanum-eco.
Preparing to unpack .../fonts-nanum-eco_1.000-6_all.deb ...
Unpacking fonts-nanum-eco (1.000-6) ...
Selecting previously unselected package fonts-nanum-extra.
Preparing to unpack .../fonts-nanum-extra_20170925-1_all.deb ...
Unpacking fonts-nanum-extra (20170925-1) ...
Selecting previously unselected package fonts-nanum-coding.
Preparing to unpack .../fonts-nanum-coding_2.5-1_all.deb ...
Unpacking fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-extra (20170925-1) ...
Setting up fonts-nanum (20170925-1) ...
Setting up fonts-nanum-coding (2.5-1) ...
Setting up fonts-nanum-eco (1.000-6) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# 한글폰트 사용 in colab
%matplotlib inline  

import matplotlib as mpl 
import matplotlib.pyplot as plt 
import matplotlib.font_manager as fm  

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' 
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False

NanumBarunGothic


In [5]:
import pandas as pd
plt.rcParams['axes.unicode_minus'] = False # 마이너스 표시
# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/데이터분석 스터디/data/data_add_score.csv')
submission = pd.read_csv('/content/drive/MyDrive/데이터분석 스터디/data/sample_submission.csv')

data['출근자수'] = data['본사정원수'] - data['본사휴가자수'] - data['본사출장자수'] - data['현본사소속재택근무자수']
data['야근비율'] = data['본사시간외근무명령서승인건수'] / data['본사정원수']

train = data.iloc[:1204] 
test = data.iloc[1204:]

train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,break_score,lunch_score,dinner_score,출근자수,야근비율
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0,4,4,2401.0,0.091503
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,0,12,8,2378.0,0.122645
2,2016-02-03,수,2601,56,180,111,0.0,1017.0,573.0,0,5,4,2365.0,0.042676
3,2016-02-04,목,2601,104,220,355,0.0,978.0,525.0,0,9,5,2277.0,0.136486
4,2016-02-05,금,2601,278,181,34,0.0,925.0,330.0,0,5,12,2142.0,0.013072


In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score

In [7]:
# 변수 선택
x_train = train[['요일', '출근자수', '야근비율', 'lunch_score', 'dinner_score']]
x_test = test[['요일', '출근자수', '야근비율', 'lunch_score', 'dinner_score']]
x_train = pd.get_dummies(x_train, columns = ['요일'])
x_test = pd.get_dummies(x_test, columns = ['요일'])
#x_train['요일'] = x_train['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
#x_test['요일'] = x_test['요일'].map({'월':0, '화':1, '수':2, '목':3, '금':4})
y1_train = train['중식계']
y2_train = train['석식계']
y1_test = test['중식계']
y2_test = test['석식계']

### GridSearchCV

In [8]:
from sklearn.model_selection import GridSearchCV,KFold

In [9]:
param_grid = {'learning_rate': [0.01,0.02,0.03,0.04],
              'subsample'    : [0.9, 0.5, 0.2, 0.1],
              'n_estimators' : [100,500,1000, 1500],
              'max_depth'    : [4,6,8,10]}

In [10]:
# 점심/저녁 식수 인원 예측할 모델 2개 생성
gbr_lunch = GradientBoostingRegressor()
gbr_dinner = GradientBoostingRegressor()

In [11]:
cv = KFold(n_splits=6)
gbr1=GridSearchCV(gbr_lunch, param_grid=param_grid, cv=cv, n_jobs=-1)
gbr2=GridSearchCV(gbr_dinner, param_grid=param_grid, cv=cv, n_jobs=-1)

In [12]:
gbr1.fit(x_train, y1_train)
print('final params', gbr1.best_params_)   # 최적의 파라미터 값 출력
print('best score', gbr1.best_score_)      # 최고의 점수

final params {'learning_rate': 0.01, 'max_depth': 8, 'n_estimators': 500, 'subsample': 0.1}
best score 0.6704566398574817


In [13]:
gbr2.fit(x_train, y2_train)
print('final params', gbr2.best_params_)   # 최적의 파라미터 값 출력
print('best score', gbr2.best_score_)      # 최고의 점수

final params {'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 500, 'subsample': 0.1}
best score 0.4206396121295586


In [14]:
# 학습
lunch_model = gbr1.best_estimator_
dinner_model = gbr2.best_estimator_

In [15]:
# 추론
pred1 = lunch_model.predict(x_test)
pred2 = dinner_model.predict(x_test)

In [16]:
# 결과 제출
submission['중식계'] = pred1
submission['석식계'] = pred2

In [17]:
submission.to_csv('/content/drive/MyDrive/데이터분석 스터디/data/GBR+feature_selection.csv', index=False)